## 최적 경로 알고리즘
### 예상 쓰레기 배출량을 이용하여 이동 우선순위 점수를 생성, 매 이동 시점마다 점수를 갱신하며 다음에 이동할 최적해를 찾는 알고리즘 설계
1. <가정> 
- 쓰레기차의 수용량은 하루에 도는 거점에서 발생하는 모든 쓰레기 양의 합보다 무조건 크다고 가정함
- 자원순환가게의 취지상 거점 하나 갔다가 - 다시 출발지점으로 가고 - 다음 거점으로 가는 과정을 반복하는 것은 말이 안됨
- 계산한 예상 쓰레기 배출량 중 극히 일부분만 자원순환가게가 수용할 것이기 때문에 위와 같이 변경함
- 운영시간은 9-6라고 가정한다. 중간 점심시간 1시간을 빼고 하루에 도는 모든 거점에 대한 소요시간은 8시간이라고 가정한다. 

- 두가지 경우 존재: 
    1. #배출량# 이 많은 곳을 먼저 가는 경우 (배출량 가중치=양수) 
    2. #배출량# 이 적은 곳을 먼저 가는 경우 (배출량 가중치=음수)  --> 이 경우로 선택
    
2. 예상 쓰레기 배출량은 해당 이동식 거점이 속하는 격자별 인구수, 분리수거함 존재 건물수, 분리수거함 미존재 건물수에 각 가중치를 곱하여 계산한다.

3. 계산된 예상 쓰레기 배출량에 일정 가중치를 곱하고, 이동해야 하는 지점 사이의 거리에 일정 마이너스 가중치를 곱해서 (거리는 짧을수록 좋음) 한 지점에서 이동할 다음 지점을 정할 때의 우선순위점수를 생성한다. 

4. 모든 노드를 다 지날 때까지 각 이동지점에서 다음에 이동할 지점을 선택할 우선순위점수를 계산하면서 모든 상황에서 그 시점의 최적해를 찾음 (가장 우선순위점수가 높은 거점으로 이동)

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings(action='ignore')


import matplotlib
from matplotlib import font_manager, rc
import platform

from pandas import Series
import math
from haversine import haversine
from numpyencoder import NumpyEncoder
import json
import openrouteservice

In [2]:
# 거점과 거점 사이의 직선거리가 아닌, 실제 차가 이동할 수 있는 이동경로 길이를 계산하기 위해 Open Route Service API 호출

client = openrouteservice.Client(key='5b3ce3597851110001cf62481e2a3fb252384e629e11b2a6a26f1b58') # Specify your personal API key

In [3]:
data = pd.read_csv('이동식_300m_0816.csv')
data

,lon,lat,버퍼당인구수,w분리수거함존재건물,w분리수거함미존재건물,순위,분리수거함존재건물,분리수거함미존재건물
0,127.056100,37.578754,128.121454,0.6,452.0,6,0,9
1,127.060044,37.588758,67.070043,0.6,340.8,7,0,5
2,127.069059,37.576935,118.079755,3.2,304.8,8,0,13
3,127.049339,37.588303,48.131236,10.6,247.2,9,1,0
4,127.049339,37.577390,58.893526,2.8,240.0,10,0,4


In [4]:
data = data[['lon', 'lat', '버퍼당인구수', '분리수거함존재건물', '분리수거함미존재건물']]

In [5]:
# data 전처리
# 출발지점은 임의로 동대문구청으로 지정 (127.03971051118135, 37.57453452544994)
# 5번 인덱스가 동대문구청임

data.loc[5] = [127.03971051118135, 37.57453452544994, 0, 0, 0]
data

,lon,lat,버퍼당인구수,분리수거함존재건물,분리수거함미존재건물
0,127.056100,37.578754,128.121454,0.0,9.0
1,127.060044,37.588758,67.070043,0.0,5.0
2,127.069059,37.576935,118.079755,0.0,13.0
3,127.049339,37.588303,48.131236,1.0,0.0
4,127.049339,37.577390,58.893526,0.0,4.0
5,127.039711,37.574535,0.000000,0.0,0.0


In [6]:
# 5번에 추가된 출발지를 0번으로 reset

tmp = []

for i in range(len(data)):
    if i ==len(data)-1:
        tmp.append(0)
    else :
        tmp.append(i+1)

data['id'] = tmp
data = data[['id', 'lon', 'lat', '버퍼당인구수', '분리수거함존재건물', '분리수거함미존재건물']]
data.sort_values(by='id',inplace=True)
data.reset_index(drop=True,inplace=True)

In [7]:
data

,id,lon,lat,버퍼당인구수,분리수거함존재건물,분리수거함미존재건물
0,0,127.039711,37.574535,0.000000,0.0,0.0
1,1,127.056100,37.578754,128.121454,0.0,9.0
2,2,127.060044,37.588758,67.070043,0.0,5.0
3,3,127.069059,37.576935,118.079755,0.0,13.0
4,4,127.049339,37.588303,48.131236,1.0,0.0
5,5,127.049339,37.577390,58.893526,0.0,4.0


In [9]:
# 노드당 예상 쓰레기 배출량 생성해서 추가
# 격자당 인구수 0.4, 분리수거함존재건물수 (-0.3), 분리수거함미존재건물수 0.3 가중치 곱해서 예상 쓰레기 배출량 변수 생성

tmp = []
for i in range(len(data)): 
    tmp.append(data['버퍼당인구수'][i] * 0.4 + data['분리수거함존재건물'][i] * (-0.3) + data['분리수거함미존재건물'][i] * (0.3))
    
data['예상 쓰레기 배출량'] = tmp
data

,id,lon,lat,버퍼당인구수,분리수거함존재건물,분리수거함미존재건물,예상 쓰레기 배출량
0,0,127.039711,37.574535,0.000000,0.0,0.0,0.000000
1,1,127.056100,37.578754,128.121454,0.0,9.0,53.948582
2,2,127.060044,37.588758,67.070043,0.0,5.0,28.328017
3,3,127.069059,37.576935,118.079755,0.0,13.0,51.131902
4,4,127.049339,37.588303,48.131236,1.0,0.0,18.952495
5,5,127.049339,37.577390,58.893526,0.0,4.0,24.757410


In [325]:
# # 우선순위 점수 계산 함수 --> 정답이 있는 문제가 아니라서 최적경로라고 할 수 없지만.. 로직은 맞다

# def cal_score(data, node_id):  # data는 원데이터, df는 방문노드 삭제한 데이터
#     score_list = []
#     score_id = []
#     score_df = pd.DataFrame()
    
#     for i in range(len(data)):   # 출발지 노드의 아이디로 참조해서 인덱스 받아오기
#         if data['id'][i] == node_id: 
#             idx = i
#             break
    
#     for i in range(1, len(data)): 
#         distance = math.sqrt((data['x'][i] - data['x'][idx])**2 + (data['y'][i] - data['y'][idx])**2) # 출발지(노드)로부터 거리
#         score_list.append(distance * (-0.4) + data['예상 쓰레기 배출량'][i] * 0.6)  # 각 가중치 곱해서 append, 거리값은 작을수록 좋음
#         score_id.append(data['id'][i])
            
#     score_df['id'] = score_id
#     score_df['score'] = score_list
    
#     idx = score_df[score_df['id'] == node_id].index
#     score_df = score_df.drop(idx)
        
#     return score_df

In [10]:
# 우선순위 점수 계산 함수 --> Open Route Service 버전으로 수정

def cal_score(data, node_id):  
    try : 
        score_list = []
        score_id = []
        score_df = pd.DataFrame()
    
        for i in range(len(data)):   # 출발지 노드의 아이디로 참조해서 인덱스 받아오기
            if data['id'][i] == node_id: 
                idx = i
                break
            
        distance_df = pd.DataFrame()
        for i in range(1, len(data)):   
            if i != idx:
                coord = ((data['lon'][i], data['lat'][i]), (data['lon'][idx], data['lat'][idx]))
                routes = client.directions(coord)
                # json_data = json.dumps(routes)  
                # distance_df = json_data
                ds =  routes['routes'][0]['summary']['distance'] # 출발지(노드)로부터 거리
                score_list.append(ds * (-0.4) + data['예상 쓰레기 배출량'][i] * (-0.6))  # 각 가중치 곱해서 append, 거리값은 작을수록 좋음
                score_id.append(data['id'][i])
            
        score_df['id'] = score_id
        score_df['score'] = score_list
    
        idx = score_df[score_df['id'] == node_id].index
        score_df = score_df.drop(idx)
        
        return score_df
    
    except:   # equests.exceptions.ConnectionError: HTTPConnectionPool 오류 해결용
        time.sleep(2)

In [11]:
# 테스트 - 0번 노드가 출발지일 때 우선순위 점수 확인

score = cal_score(data, 0)
score

,id,score
0,1,-1016.929149
1,2,-1599.436810
2,3,-1445.599141
3,4,-1088.891497
4,5,-562.774446


In [12]:
# 모든 노드에 대한 이동경로 생성

lst = []
st_id = 0
k = data['id'].tolist()  # 모든 노드에 대한 아이디 리스트 생성
df = data.copy()

while len(k) > 0:  # 모든 노드를 방문할 동안
    
    lst.append(st_id)  # 이동경로에 노드 더하기
    k.remove(st_id)   # 방문한 노드 빼기 (하나씩 카운트)
    
    score_ = cal_score(df, st_id)
    score_ = score_.sort_values(by = 'score', ascending = False).reset_index(drop=True)  # 내림차순
    # print(score_)
    
    for i in score_['id']:    # score 중 가장 점수가 높고, 아직 방문하지 않은 노드의 아이디 가져오기
        if i in k: 
            st_id = i
            break
    
for i in range(len(lst)):  # 예쁘게 출력
    if i == len(lst)-1:
        print(lst[i])
    else :
        print(lst[i],end=' -> ')

0 -> 5 -> 1 -> 3 -> 2 -> 4


In [13]:
lst

[0, 5, 1, 3, 2, 4]

In [14]:
lst_df = pd.DataFrame(lst)
lst_df.rename(columns = {0 : 'id'}, inplace = True)

In [15]:
lst_df

,id
0,0
1,5
2,1
3,3
4,2
5,4


In [16]:
sorted_df = pd.merge(lst_df, data, on='id')
sorted_df

,id,lon,lat,버퍼당인구수,분리수거함존재건물,분리수거함미존재건물,예상 쓰레기 배출량
0,0,127.039711,37.574535,0.000000,0.0,0.0,0.000000
1,5,127.049339,37.577390,58.893526,0.0,4.0,24.757410
2,1,127.056100,37.578754,128.121454,0.0,9.0,53.948582
3,3,127.069059,37.576935,118.079755,0.0,13.0,51.131902
4,2,127.060044,37.588758,67.070043,0.0,5.0,28.328017
5,4,127.049339,37.588303,48.131236,1.0,0.0,18.952495


In [17]:
sorted_df.to_csv('이동식_최적경로_최종.csv', encoding = 'euc-kr')